In [1]:
import json
import os
import glob
os.chdir('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/students/tests/sprint2') # path to the students repo

In [2]:
!git pull

Already up to date.


In [3]:
student_deets = json.load(open('/media/sanketm/Data/Temp/CS314/gradingandscripts/people.json','r'))
student_deets[0]

{'github': 'davemattcsu',
 'team': 't00',
 'name': 'Dave Matthews',
 'eid': 'davematt',
 'id': 1,
 'section': '001',
 'dev': '34100',
 'sprint': '31400',
 'language': 'java'}

In [4]:
student_eid = [a['eid'] for a in student_deets]
#student_eid

In [5]:
all_jsons = glob.glob('./**.json') # get list of all json files

In [6]:
student_dict = {}

for file in all_jsons:
    with open(file, 'r') as f:
        try:
            eid = file.split("-")[0].split("./")[1]# splitting. / shlok - distance.json to get shlok
            req = file.split("-")[1].split(".")[0]# splitting. / shlok - distance.json to get distance
            eid = eid.lower()
            req = req.lower()

            student_rec = student_dict.get(eid, {})

            if req == 'find':
                print(f"{eid} has find")
                student_rec['submitted find'] = 1
                student_rec['find_filename'] = file
#             else:
#                 student_rec['submitted find'] = 0
            
            if req == 'distance':
                print(f"{eid} has distance")
                student_rec['submitted distance'] = 1
                student_rec['distance_filename'] = file
#             else:
#                 student_rec['submitted distance'] = 0
                
            json_file = json.load(f)      
            if json_file:
                if req == 'find':
                    student_rec['valid find'] = 1
#                 else:
#                     student_rec['valid find'] = 0
                    
                if req == 'distance':
                    student_rec['valid distance'] = 1
#                 else:
#                     student_rec['valid distance'] = 0
                    
                if req not in ['find','distance']:
                    student_rec['invalid_filename'] = 1
                    student_rec['comment'] = "Student's filename: "+ file +"was not in the format eid_find/distance, so was not valid." 
                    student_rec['invalid_filename'] = file
            student_dict[eid] = student_rec
        except:
            print(f"{file} is invalid ")
            
            eid = file.split("-")[0].split("./")[1]# splitting. / shlok - distance.json to get shlok
            req = file.split("-")[1].split(".")[0]# splitting. / shlok - distance.json to get distance
            eid = eid.lower()
            req = req.lower()

            student_rec = student_dict.get(eid, {})
            
            if req == 'find':
                student_rec['submitted find'] = 1
            else:
                student_rec['submitted find'] = 0
                
            if req == 'distance':
                student_rec['submitted distance'] = 1
            else:
                student_rec['submitted distance'] = 0
            print(student_rec)
            student_dict[eid] = student_rec
            #continue


aericjw has distance
aericjw has find
ah00t13 has distance
ah00t13 has find
alecmilt has distance
alecmilt has find
augustai has distance
augustai has find
backlira has distance
backlira has find
barelads has distance
barelads has find
bcombs18 has distance
bcombs18 has find
benschra has distance
benschra has find
bobbycsu has find
braedens has distance
braedens has find
camconn has distance
camconn has find
caustin has distance
caustin has find
cessna17 has distance
cessna17 has find
ch85 has distance
ch85 has find
chenhui has distance
chenhui has find
cmar1299 has distance
cmar1299 has find
colecole has distance
connorsw has distance
connorsw has find
criech5 has distance
criech5 has find
cwernsma has distance
cwernsma has find
darinh has distance
darinh has find
davematt has distance
dbilbao has distance
dbilbao has find
deionus has distance
deionus has find
drew21 has distance
drew21 has find
eberleben15 has distance
edvarela has distance
edvarela has find
eyreda has find
fonsecao 

In [7]:
for name in student_eid:
    if name not in student_dict.keys():
        student_rec = {}
        student_rec['submitted find'] = 0
        student_rec['submitted distance'] = 0
        student_dict[name] = student_rec
    

In [8]:
print(student_dict)

{'aericjw': {'submitted distance': 1, 'distance_filename': './aericjw-distance.json', 'valid distance': 1, 'submitted find': 1, 'find_filename': './aericjw-find.json', 'valid find': 1}, 'ah00t13': {'submitted distance': 1, 'distance_filename': './ah00t13-distance.json', 'valid distance': 1, 'submitted find': 1, 'find_filename': './ah00t13-find.json', 'valid find': 1}, 'alecmilt': {'submitted distance': 1, 'distance_filename': './alecmilt-distance.json', 'valid distance': 1, 'submitted find': 1, 'find_filename': './alecmilt-find.json', 'valid find': 1}, 'augustai': {'submitted distance': 1, 'distance_filename': './augustai-distance.json', 'valid distance': 1, 'submitted find': 1, 'find_filename': './augustai-find.json', 'valid find': 1}, 'backlira': {'submitted distance': 1, 'distance_filename': './backlira-distance.json', 'valid distance': 1, 'submitted find': 1, 'find_filename': './backlira-find.json', 'valid find': 1}, 'barelads': {'submitted distance': 1, 'distance_filename': './bar

In [9]:
len(student_dict)

126

In [15]:
for key in ['cs314','kaseda','sanketm','zhouxy','dzuigas','dziugas']:
    if student_dict.get(key,[]) != []:
        del(student_dict[key])

In [16]:
len(student_dict)

121

In [17]:
import pandas as pd
df = pd.DataFrame().from_dict(student_dict)
df = df.fillna(0)
df = df.transpose()

In [18]:
os.chdir('/media/sanketm/Data/Temp/CS314/gradingandscripts')
df.to_csv('eid_sprint2_tests.csv')

In [14]:
# with open('eid_sprint2_tests.csv', 'w') as f:
#     for key in student_dict.keys():
#         f.write("%s,%s\n"%(key,student_dict[key]))